# Exemplo de extração de dados de páginas web

# https://www.vivareal.com.br/venda/?pagina=5

Este notebook extrai os dados de todos os [imóveis a venda no site VivaReal](https://www.vivareal.com.br/venda/) e escreve os dados extraidos num banco de dados (SQLIte).

![cartao-vivareal.png](cartao-vivareal.png)

Na última verificação, o site tinha 277 páginas de imóveis, 36 imóveis por página. Se deixá-lo rodar integralmente, teremos no final quase 10000 imóveis.

Notebook escrito por Avi Alkalay <<DataScientist@digitalhouse.com>>

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sqlite3

In [ ]:
# 2 opções de URL
urlBrasil = "https://www.vivareal.com.br/venda/?pagina={npagina}"
urlPinheiros = "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-oeste/pinheiros/apartamento_residencial/?pagina={npagina}"

# Vamos trabalhar com esta:
url=urlBrasil

# Para enganar o site, permutaremos entre 2 opções de assinaturas de browser.
# Peguei de https://developers.whatismybrowser.com/
userAgents=[
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/74.0.3729.157 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15"
]

dbfile = 'VivaReal.db'
tabela = 'venda'

## Análise inicial de cada elemento da página

Este trecho serve para testarmos a extração de cada elemento desejado, seja nome do imóvel, preço etc. Quando compreendido, movemos o trecho de código finalizado para o bloco mais adiante no notebook, em que o scrapping é feito em lote.

### Passo 1: Traz o HTML do site

Use o `requests` para fazer uma operação HTTP GET e obter o HTML da página.

In [ ]:
doc = requests.get(url.format(npagina=1), headers={"User-agent": userAgents[1]})

Vamos verificar o que o webserver retornou olhando só os 700 primeiros bytes (variavel `olhadinha`).

In [ ]:
olhadinha=700
print(str(doc.content[:olhadinha]) + '…')

In [ ]:
doc.status_code

### Passo 2: Analiza o texto HTML

A biblioteca `BeautifulSoup` tem a capacidade de analizar o HTML entregue pelo site (que não é nada mais que texto corrido, como vimos), e convertê-lo em DOM (document object model). O DOM é o mesmo documento HTML só que todas as tags e sua hierarquia foram identificadas ao ponto de podermos fazer buscas por tags e attributos de seus tags.

In [ ]:
analizador = BeautifulSoup(doc.content, 'html.parser')

A variável `analizador` agora contém o DOM do texto retornado pelo web server. Então agora usamos ela para buscar tags específicas.

Usamos o _inspector_ do browser para entender a estrutura do documento que o HTMLeiro da VivaReal concebeu. Descobrimos que os imóveis estão publicados assim:

```html
<div id="12345">
    <div class="js-card-selector">
        ...mais tags sobre o imóvel...
    </div>
</div>

<div id="54321">
    <div class="js-card-selector">
        ...mais tags sobre o imóvel...
    </div>
</div>
```

Cada imóvel está contido dentro de seu respectivo `<div id="....">`, mas não temos nada específico e genérico nesta tag para selecioná-la. Já a tag imediatamente interior, `<div class="js-card-selector">`, é idêntica para cada imóvel e contém a classe `js-card-selector`, o que a torna uma ótima candidata para ser selecionada.

Então abaixo criaremos uma lista chamada `imoveis` que contém todos os trechos DOM que respeitam o nosso filtro: `<div>`s com classe `js-card-selector`.

In [ ]:
analizador

In [ ]:
imoveis = analizador.find_all('div', class_="js-card-selector")

Vamos dar uma olhadinha no que tem dentro de cada ítem do array:

In [ ]:
for i in imoveis[:5]:
    print(str(i).replace("\n","").replace("   ","")[:200] + "…\n")

In [ ]:
len(imoveis)

### Passo 3: Monto receitas para extrair cada dado que desejo

Ótimo. Agora olhando a estrutura de tags e atributos, uso seletores para isolar exatamente o dado que desejo. Após isolá-lo, ainda forço ele a passar por filtros para eliminar inutilidades como espaços (uso `strip()`), prefixos como "R$ " (uso `replace("R$ ","")`) e ajusto o ponto decimal de números (uso `replace(".","")`) para em seguida convertê-los de texto para números (uso `int()`).

É muito importante para mim também obter o ID do imóvel. Como esta informação está num atrbuto do `<div>` pai, uso o seletor `parent`.

In [ ]:
# Título do anúncio
print(imoveis[0].find("span",class_="property-card__title").contents[0].strip())

In [ ]:
# Endereço
print(imoveis[0].find("span",class_="property-card__address").contents[0].strip())

In [ ]:
imoveis[0].find("div",class_="property-card__price").find("p").contents[0].strip().replace("R$ ","").replace(".","")

In [ ]:
float(imoveis[0].find("div",class_="property-card__price").find("p").contents[0].strip().replace("R$ ","").replace(".",""))

In [ ]:
# Preço limpo e transformado em inteiro
print(int(imoveis[0].find("div",class_="property-card__price").find("p").contents[0].strip().replace("R$ ","").replace(".","")))

In [ ]:
# Telefone
try:
    print(imoveis[0].find("a",class_="property-card__contact--phone").get('href').replace("tel:","+55"))
except:
    pass

In [ ]:
# Texto descritivo
try:
    print(imoveis[0].find('ul',class_='property-card__details').text.strip())
except:
    pass

In [ ]:
print(int(imoveis[0].find('ul',class_='property-card__details').find('span',class_='property-card__detail-area').text.strip()))

In [ ]:
# imoveis[0].find('div',class_='property-card__description').contents[0].strip()

# ID do imovel
print(imoveis[0].parent.get('id'))

## Extração em massa

Depois da análise acima, podemos criar nosso loop que consome todo o site. Página por página, imóvel por imóvel, dado por dado.

In [ ]:
# são 277 páginas de imóveis a venda no total
# paginas=277

# mas a títlo de exemplo, usaremos somente 20 páginas:
paginas=20

In [ ]:
# em rows guardo uma lista temporária de dicts de dados capturados, 1 dict por imóvel
imóveis=[]

for pagina in range(1,paginas+1):
    print("Estou aqui: " + url.format(npagina=pagina))
    
    # pega a página do site pela internet
    doc = requests.get(url.format(npagina=pagina))
    
    # analiza o HTML
    analizador = BeautifulSoup(doc.content, 'html.parser')
    
    # extrai somente a lista de imóveis (em HTML) usando o seletor descoberto no código da página
    imoveis = analizador.find_all('div', class_="js-card-selector")
    
    for unidade in imoveis:
        uni={}
        # extrai dado por dado segundo seus seletores...
        
        # O ID:
        uni['id']    = unidade.parent.get('id')
        
        # O título/nome:
        uni['nome']  = unidade.find("span",class_="property-card__title").contents[0].strip()
        
        # O endereço:
        uni['ende']  = unidade.find("span",class_="property-card__address").contents[0].strip()
        
        # A metragem
        uni['metragem']=int(unidade.find('ul',class_='property-card__details').find('span',class_='property-card__detail-area').text.strip())
        
        # O preço tem uma pegadinha
        try:
            # Na maioria dos anuncios o preço está na posição 0.
            # Mas em alguns, o preço aparece como "a partir de" e causa um erro aqui.
            uni['preco'] = int(unidade.find("div",class_="property-card__price").find("p").contents[0].strip().replace("R$ ","").replace(".",""))
        except ValueError:
            # Então o meu tratamento de erro para preços "a partir de" é tentar capturá-lo na posição 2:
            try:
                uni['preco'] = int(unidade.find("div",class_="property-card__price").contents[2].strip().replace("R$ ","").replace(".",""))
            except IndexError:
                # Mas há um outro tipo de problema, onde o preço aparece "sob consulta", e causa um "IndexError". Trato assim:
                uni['preco'] = -1

        # O telefone capturo manipulando a URL e dando uma internacionalizada básica com "+55":
        try:
            uni['tel']   = unidade.find("a",class_="property-card__contact--phone").get('href').replace("tel:","+55")
        except:
            pass
        
        # A descrição:
        try:
            uni['desc']  = unidade.find('ul',class_='property-card__details').text.strip()
        except:
            pass

        # No final deste loop, o dict uni contém os dados de 1 imóvel, aí adiciono-o a uma lista de imóveis
        imóveis.append(uni)

Processei todos os imóveis de todas as páginas. Vamos ver o resultado...

In [ ]:
imóveis

Agora converterei minha lista de dicts para um DataFrame chamado `todosOsImoveis`. Para tal, crio um DataFrame vazio nomeando as colunas com as chaves de um dict `uni`.

In [ ]:
# Crio um DataFrame vazio cujas colunas são as chaves de 1 unidade:
todosOsImoveis=pd.DataFrame(columns=uni.keys())
todosOsImoveis

Com o esqueleto do DataFrame pronto, adiciono nele todos os meus dicts (imóveis).

In [ ]:
todosOsImoveis=todosOsImoveis.append(imóveis)

todosOsImoveis.head()

Agora em Pandas, vamos dar uma otimizadinha básica convertendo algumas colunas textuais para tipos numéricos.

In [ ]:
todosOsImoveis.info()

Como ficaram os tipos de nossas colunas...

In [ ]:
todosOsImoveis=todosOsImoveis.convert_dtypes()

In [ ]:
todosOsImoveis.info()

Por que a coluna `id` não foi convertida para `Int64` como as outras? Vamos forçar:

In [ ]:
todosOsImoveis['id']=pd.to_numeric(todosOsImoveis['id'])

In [ ]:
todosOsImoveis.info()

Seta o índice para o ID do imóvel

In [ ]:
todosOsImoveis.set_index(keys='id', inplace=True)

Eis o nosso DataFrame pronto:

In [ ]:
todosOsImoveis.head()

## Grava o DataFrame inteiro num Banco de Dados

Este é o código para abrir ou criar um aquivo SQLite chamado `VivaReal.db`, na tabela `venda`. Para usar outro banco, como MariaDB, Oracle, DB2, altere esta célula e use a biblioteca SQL Alchemy com os drivers corretos. Todo o resto do código funcionará igual.

In [ ]:
dbfile = 'VivaReal.db'
tabela = 'venda'

db = sqlite3.connect(dbfile)

Determina os tipos de cada coluna SQL baseado nos tipos das colunas Pandas:

In [ ]:
sqlDataTypes={}

for c in todosOsImoveis.columns:
    if todosOsImoveis[c].dtype.kind == 'i':
        sqlDataTypes[c]='INTEGER'
    elif todosOsImoveis[c].dtype.kind == 'f':
        sqlDataTypes[c]='REAL'
    else:
        sqlDataTypes[c]='TEXT'

sqlDataTypes

Escreve na tabela

In [ ]:
todosOsImoveis.to_sql(tabela, index=True, if_exists='replace', dtype=sqlDataTypes, con=db)

Efetiva as operações no DB e fecha o arquivo

In [ ]:
db.commit()
db.close()

## Checa os dados no DB

In [ ]:
dbfile = 'VivaReal.db'
tabela = 'venda'

db = sqlite3.connect(dbfile)

imoveisCaros = pd.read_sql_query(f'select * from "{tabela}" where preco>600000', db)

imoveisCaros

## Sincronização Diária

Para manter os dados em dia, este procedimento deve ser executado diariamente.

No final da extração comparo todos os imóveis da minha base histórica com todos os imóveis que acabei de extrair novamente. ID por ID. É por isso que é tão importante se prender aos IDs. É neste momento que detecto qual imóvel sumiu da base, qual apareceu, qual mudou de preço ou descrição.

* Imóveis que estão na minha base histórica mas não estão na última extração serão marcados como deletados ou “unlisted”.
* Imóveis que não estavam na minha base histórica e que acabaram de aparecer na última extração serão marcados como novos.
* Imóveis que aprentam preços diferentes entre a base histórica e a última extração devem ter essa alteração registrada em outra tabela. É interessante manter esse histórico de alteração de preços de imóveis.

Certamente nossa modelagem simples de tabela não suporta registrar a mudança de preços. **Fica de lição** de casa criar uma tabela separada de preços de imóveis para que se possa manter um histórico.

## Exercício

Melhorar o código e extrair também a lista de amenidades (elevador, piscina, churrasqueira etc)